In [1]:
import numpy as np
import pandas as pd

In [2]:
from dmbtools.wellbore.survey import position_log, position_log_meta
from dmbtools.wellbore.horizontal_stats_pak import h_stats, h_stats_meta
from dmbtools.geodetic.projection import GridConvergence
from dmbtools.geodetic import convert as cvt
from dmbtools.geodetic import projection as prj
from dmbtools.math import utils as mutils

In [3]:
tie_ins = (pd.read_csv(r'../h_stats/data/TDM_UI_DirSurveys_COGEMP_isPreferred_20190307_tie_ins.csv', low_memory=False)
             .set_index(['ROW_KEY']))

In [4]:
surveys = (pd.read_csv(r'../h_stats/data/TDM_UI_DirSurveys_COGEMP_isPreferred_20190307_surveys.csv', low_memory=False)
             .set_index(['ROW_KEY']))

In [5]:
sorces = ['PI DIRECTIONAL SURVEYS', 'COG EMPLOYEE', 'TGS', 'IHS', 'COG AUDIT']

In [6]:
tie_ins[tie_ins.SOURCE == sorces[1]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1882 entries, COG EMPLOYEE:10137402-003:003:0 to COG EMPLOYEE:14580176-001:001:0
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GOVERNMENT_ID  1882 non-null   int64  
 1   SOURCE         1882 non-null   object 
 2   SRVY_KEY       1882 non-null   object 
 3   STATION_MD     1882 non-null   float64
 4   LAT_SURFACE    1882 non-null   float64
 5   LONG_SURFACE   1882 non-null   float64
 6   N_OFFSET       1882 non-null   float64
 7   E_OFFSET       1882 non-null   float64
 8   STATION_TVD    1882 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 147.0+ KB


In [7]:
tie_ins[tie_ins.SOURCE == sorces[-1]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, COG AUDIT:10480700-001:002:0 to COG AUDIT:14464823-001:002:0
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GOVERNMENT_ID  6 non-null      int64  
 1   SOURCE         6 non-null      object 
 2   SRVY_KEY       6 non-null      object 
 3   STATION_MD     6 non-null      float64
 4   LAT_SURFACE    6 non-null      float64
 5   LONG_SURFACE   6 non-null      float64
 6   N_OFFSET       6 non-null      float64
 7   E_OFFSET       6 non-null      float64
 8   STATION_TVD    6 non-null      float64
dtypes: float64(6), int64(1), object(2)
memory usage: 480.0+ bytes


#### Get a survey key of one survey

In [8]:
srvy_key = tie_ins[tie_ins.SOURCE == sorces[-1]]['SRVY_KEY'][0]
srvy_key

'10480700-001:002'

In [9]:
gov_id = tie_ins[tie_ins.SOURCE == sorces[-1]]['GOVERNMENT_ID'][0]
gov_id

421093261000

In [10]:
srv = surveys.loc[surveys.SRVY_KEY == srvy_key].copy()
srv

SOURCE          SRVY_KEY  STATION_MD  \
ROW_KEY                                                                       
COG AUDIT:10480700-001:002:0        COG AUDIT  10480700-001:002         0.0   
COG AUDIT:10480700-001:002:10000    COG AUDIT  10480700-001:002       100.0   
COG AUDIT:10480700-001:002:20000    COG AUDIT  10480700-001:002       200.0   
COG AUDIT:10480700-001:002:30000    COG AUDIT  10480700-001:002       300.0   
COG AUDIT:10480700-001:002:40000    COG AUDIT  10480700-001:002       400.0   
...                                       ...               ...         ...   
COG AUDIT:10480700-001:002:1255400  COG AUDIT  10480700-001:002     12554.0   
COG AUDIT:10480700-001:002:1265000  COG AUDIT  10480700-001:002     12650.0   
COG AUDIT:10480700-001:002:1274500  COG AUDIT  10480700-001:002     12745.0   
COG AUDIT:10480700-001:002:1279100  COG AUDIT  10480700-001:002     12791.0   
COG AUDIT:10480700-001:002:1283400  COG AUDIT  10480700-001:002     12834.0   

                                    INCLINATION  AZIMUTH  LAT_SURFACE  \
ROW_KEY                                                                 
COG AUDIT:10480700-001:002:0               0.00   358.07    31.971339   
COG AUDIT:10480700-001:002:10000           0.39   335.59    31.971339   
COG AUDIT:10480700-001:002:20000           0.53   330.41    31.971339   
COG AUDIT:10480700-001:002:30000           1.15   326.45    31.971339   
COG AUDIT:10480700-001:002:40000           1.77   322.49    31.971339   
...                                         ...      ...          ...   
COG AUDIT:10480700-001:002:1255400        91.00   358.47    31.971339   
COG AUDIT:10480700-001:002:1265000        91.60   358.17    31.971339   
COG AUDIT:10480700-001:002:1274500        89.50   358.57    31.971339   
COG AUDIT:10480700-001:002:1279100        89.50   358.07    31.971339   
COG AUDIT:10480700-001:002:1283400        89.50   358.07    31.971339   

                                    LONG_SURFACE  DELTA_LAT  DELTA_LONG  
ROW_KEY                                                                  
COG AUDIT:10480700-001:002:0          -104.08486   0.000000    0.000000  
COG AUDIT:10480700-001:002:10000      -104.08486   0.000001    0.000000  
COG AUDIT:10480700-001:002:20000      -104.08486   0.000003   -0.000002  
COG AUDIT:10480700-001:002:30000      -104.08486   0.000006   -0.000004  
COG AUDIT:10480700-001:002:40000      -104.08486   0.000012   -0.000009  
...                                          ...        ...         ...  
COG AUDIT:10480700-001:002:1255400    -104.08486   0.012702   -0.000031  
COG AUDIT:10480700-001:002:1265000    -104.08486   0.012966   -0.000040  
COG AUDIT:10480700-001:002:1274500    -104.08486   0.013227   -0.000049  
COG AUDIT:10480700-001:002:1279100    -104.08486   0.013353   -0.000053  
COG AUDIT:10480700-001:002:1283400    -104.08486   0.013471   -0.000058  

[155 rows x 9 columns]

In [11]:
srv['LAT'] = (srv['LAT_SURFACE'] + srv['DELTA_LAT'])

In [12]:
srv['LONG'] = (srv['LONG_SURFACE'] + srv['DELTA_LONG'])

#### A dataset that has the TDM calc'ed lat/lon's

In [13]:
srv

SOURCE          SRVY_KEY  STATION_MD  \
ROW_KEY                                                                       
COG AUDIT:10480700-001:002:0        COG AUDIT  10480700-001:002         0.0   
COG AUDIT:10480700-001:002:10000    COG AUDIT  10480700-001:002       100.0   
COG AUDIT:10480700-001:002:20000    COG AUDIT  10480700-001:002       200.0   
COG AUDIT:10480700-001:002:30000    COG AUDIT  10480700-001:002       300.0   
COG AUDIT:10480700-001:002:40000    COG AUDIT  10480700-001:002       400.0   
...                                       ...               ...         ...   
COG AUDIT:10480700-001:002:1255400  COG AUDIT  10480700-001:002     12554.0   
COG AUDIT:10480700-001:002:1265000  COG AUDIT  10480700-001:002     12650.0   
COG AUDIT:10480700-001:002:1274500  COG AUDIT  10480700-001:002     12745.0   
COG AUDIT:10480700-001:002:1279100  COG AUDIT  10480700-001:002     12791.0   
COG AUDIT:10480700-001:002:1283400  COG AUDIT  10480700-001:002     12834.0   

                                    INCLINATION  AZIMUTH  LAT_SURFACE  \
ROW_KEY                                                                 
COG AUDIT:10480700-001:002:0               0.00   358.07    31.971339   
COG AUDIT:10480700-001:002:10000           0.39   335.59    31.971339   
COG AUDIT:10480700-001:002:20000           0.53   330.41    31.971339   
COG AUDIT:10480700-001:002:30000           1.15   326.45    31.971339   
COG AUDIT:10480700-001:002:40000           1.77   322.49    31.971339   
...                                         ...      ...          ...   
COG AUDIT:10480700-001:002:1255400        91.00   358.47    31.971339   
COG AUDIT:10480700-001:002:1265000        91.60   358.17    31.971339   
COG AUDIT:10480700-001:002:1274500        89.50   358.57    31.971339   
COG AUDIT:10480700-001:002:1279100        89.50   358.07    31.971339   
COG AUDIT:10480700-001:002:1283400        89.50   358.07    31.971339   

                                    LONG_SURFACE  DELTA_LAT  DELTA_LONG  \
ROW_KEY                                                                   
COG AUDIT:10480700-001:002:0          -104.08486   0.000000    0.000000   
COG AUDIT:10480700-001:002:10000      -104.08486   0.000001    0.000000   
COG AUDIT:10480700-001:002:20000      -104.08486   0.000003   -0.000002   
COG AUDIT:10480700-001:002:30000      -104.08486   0.000006   -0.000004   
COG AUDIT:10480700-001:002:40000      -104.08486   0.000012   -0.000009   
...                                          ...        ...         ...   
COG AUDIT:10480700-001:002:1255400    -104.08486   0.012702   -0.000031   
COG AUDIT:10480700-001:002:1265000    -104.08486   0.012966   -0.000040   
COG AUDIT:10480700-001:002:1274500    -104.08486   0.013227   -0.000049   
COG AUDIT:10480700-001:002:1279100    -104.08486   0.013353   -0.000053   
COG AUDIT:10480700-001:002:1283400    -104.08486   0.013471   -0.000058   

                                          LAT        LONG  
ROW_KEY                                                    
COG AUDIT:10480700-001:002:0        31.971339 -104.084860  
COG AUDIT:10480700-001:002:10000    31.971340 -104.084860  
COG AUDIT:10480700-001:002:20000    31.971342 -104.084862  
COG AUDIT:10480700-001:002:30000    31.971345 -104.084864  
COG AUDIT:10480700-001:002:40000    31.971351 -104.084869  
...                                       ...         ...  
COG AUDIT:10480700-001:002:1255400  31.984041 -104.084891  
COG AUDIT:10480700-001:002:1265000  31.984305 -104.084900  
COG AUDIT:10480700-001:002:1274500  31.984566 -104.084909  
COG AUDIT:10480700-001:002:1279100  31.984692 -104.084913  
COG AUDIT:10480700-001:002:1283400  31.984810 -104.084918  

[155 rows x 11 columns]

In [14]:
tie_in = tie_ins[tie_ins.SRVY_KEY == srvy_key][['N_OFFSET', 'E_OFFSET', 'STATION_TVD']].to_numpy()[0]
tie_in

array([0., 0., 0.])

In [15]:
stations = srv[['STATION_MD', 'INCLINATION', 'AZIMUTH']].to_numpy()
stations

array([[0.0000e+00, 0.0000e+00, 3.5807e+02],
       [1.0000e+02, 3.9000e-01, 3.3559e+02],
       [2.0000e+02, 5.3000e-01, 3.3041e+02],
       [3.0000e+02, 1.1500e+00, 3.2645e+02],
       [4.0000e+02, 1.7700e+00, 3.2249e+02],
       [5.0000e+02, 1.8200e+00, 3.2028e+02],
       [6.0000e+02, 1.8800e+00, 3.1806e+02],
       [7.0000e+02, 1.4300e+00, 3.0117e+02],
       [8.0000e+02, 9.7000e-01, 2.8428e+02],
       [9.0000e+02, 7.2000e-01, 3.0269e+02],
       [1.0000e+03, 4.7000e-01, 3.2111e+02],
       [1.1000e+03, 5.7000e-01, 3.1417e+02],
       [1.2000e+03, 6.6000e-01, 3.0723e+02],
       [1.3000e+03, 9.6000e-01, 2.8899e+02],
       [1.4000e+03, 1.2500e+00, 2.7075e+02],
       [1.5000e+03, 1.4700e+00, 2.6788e+02],
       [1.6000e+03, 1.6600e+00, 2.6502e+02],
       [1.7000e+03, 1.6600e+00, 2.6900e+02],
       [1.8000e+03, 1.6700e+00, 2.9498e+02],
       [1.9000e+03, 1.2500e+00, 3.5199e+02],
       [2.0000e+03, 8.3000e-01, 3.8990e+01],
       [2.1000e+03, 1.6700e+00, 5.7490e+01],
       [2.

#### calc the position log

In [16]:
plog = position_log(stations, tie_in, report_raw=False)

In [17]:
df = pd.DataFrame(plog, columns=position_log_meta()['names'])
df

md    inc     azi        pos_n      pos_e        pos_v   dog_leg
0        0.0   0.00  358.07     0.000000   0.000000     0.000000  0.000000
1      100.0   0.39  335.59     0.309916  -0.140649    99.999228  0.390000
2      200.0   0.53  330.41     1.022017  -0.509679   199.995984  0.145905
3      300.0   1.15  326.45     2.260533  -1.292664   299.984757  0.622342
4      400.0   1.77  322.49     4.321940  -2.787643   399.951830  0.627788
..       ...    ...     ...          ...        ...          ...       ...
150  12554.0  91.00  358.47  4620.791596  -9.689044  8122.137706  0.613770
151  12650.0  91.60  358.17  4716.725085 -12.502738  8119.959727  0.698735
152  12745.0  89.50  358.57  4811.676762 -15.204676  8119.047853  2.250261
153  12791.0  89.50  358.07  4857.655093 -16.553218  8119.449277  1.086915
154  12834.0  89.50  358.07  4900.629063 -18.001337  8119.824518  0.000000

[155 rows x 7 columns]

In [18]:
nev_off = plog[:, 3:6]
nev_off

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.09915612e-01, -1.40649183e-01,  9.99992278e+01],
       [ 1.02201660e+00, -5.09678737e-01,  1.99995984e+02],
       [ 2.26053318e+00, -1.29266426e+00,  2.99984757e+02],
       [ 4.32193958e+00, -2.78764349e+00,  3.99951830e+02],
       [ 6.76844650e+00, -4.74278957e+00,  4.99902769e+02],
       [ 9.21002636e+00, -6.85387653e+00,  5.99850654e+02],
       [ 1.10760126e+01, -9.01785377e+00,  6.99809270e+02],
       [ 1.19306296e+01, -1.09058058e+01,  7.99787374e+02],
       [ 1.24787579e+01, -1.22548833e+01,  8.99776601e+02],
       [ 1.31373409e+01, -1.30411693e+01,  9.99771218e+02],
       [ 1.38031714e+01, -1.36554519e+01,  1.09976710e+03],
       [ 1.44982187e+01, -1.44708084e+01,  1.19976134e+03],
       [ 1.51192740e+01, -1.57215154e+01,  1.29975139e+03],
       [ 1.54061492e+01, -1.76043023e+01,  1.39973300e+03],
       [ 1.53729771e+01, -1.99767546e+01,  1.49970478e+03],
       [ 1.51997930e+01, -2.27015088e+01

In [19]:
nev = cvt.feet2meters(nev_off)
nev

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 9.44622787e-02, -4.28698709e-02,  3.04797646e+01],
       [ 3.11510660e-01, -1.55350079e-01,  6.09587760e+01],
       [ 6.89010513e-01, -3.94004067e-01,  9.14353540e+01],
       [ 1.31732718e+00, -8.49673735e-01,  1.21905318e+02],
       [ 2.06302249e+00, -1.44560226e+00,  1.52370364e+02],
       [ 2.80721604e+00, -2.08906157e+00,  1.82834479e+02],
       [ 3.37596864e+00, -2.74864183e+00,  2.13301866e+02],
       [ 3.63645591e+00, -3.32408961e+00,  2.43775192e+02],
       [ 3.80352539e+00, -3.73528842e+00,  2.74251908e+02],
       [ 4.00426151e+00, -3.97494840e+00,  3.04730267e+02],
       [ 4.20720663e+00, -4.16218175e+00,  3.35209011e+02],
       [ 4.41905706e+00, -4.41070242e+00,  3.65687256e+02],
       [ 4.60835472e+00, -4.79191790e+00,  3.96164225e+02],
       [ 4.69579429e+00, -5.36579134e+00,  4.26638617e+02],
       [ 4.68568343e+00, -6.08891481e+00,  4.57110016e+02],
       [ 4.63289691e+00, -6.91941989e+00

In [20]:
lat_lon = tie_ins[tie_ins.SRVY_KEY == srvy_key][['LAT_SURFACE', 'LONG_SURFACE']].to_numpy()[0]

#### Calc lat/lon using LMP

In [21]:
lat_lon

array([  31.971339, -104.08486 ])

In [22]:
lmp = cvt.lmp(nev, lat_lon[0], lat_lon[1])
lmp

array([[  31.971339  , -104.08486   ],
       [  31.97133985, -104.08486045],
       [  31.97134181, -104.08486164],
       [  31.97134521, -104.08486417],
       [  31.97135088, -104.08486899],
       [  31.97135761, -104.08487529],
       [  31.97136432, -104.0848821 ],
       [  31.97136945, -104.08488908],
       [  31.9713718 , -104.08489517],
       [  31.9713733 , -104.08489952],
       [  31.97137511, -104.08490205],
       [  31.97137694, -104.08490403],
       [  31.97137885, -104.08490666],
       [  31.97138056, -104.0849107 ],
       [  31.97138135, -104.08491677],
       [  31.97138126, -104.08492442],
       [  31.97138078, -104.08493321],
       [  31.97138037, -104.08494253],
       [  31.97138199, -104.08495146],
       [  31.97138665, -104.08495621],
       [  31.97139117, -104.08495523],
       [  31.97139487, -104.0849498 ],
       [  31.97139948, -104.08493936],
       [  31.97140364, -104.08492866],
       [  31.97140623, -104.08492234],
       [  31.97140787, -1

#### Diff from TDM

In [23]:
print(srv.iloc[-1][['LAT', 'LONG']].to_numpy() - lmp[-1])

[-5.321272556813028e-06 5.9402438523648016e-08]


#### Calc lat/lon's using inverse projection

In [24]:
cart = cvt.api_num_2_epsg([str(gov_id)]).iloc[0]
cart

32039

In [25]:
geog = cvt.Datums().epsg
geog

4267

#### Forward rotate the azimuths to grid north

In [26]:
azi_grid_north = prj.toTrueNorthAZI(stations[:, -1], lat_lon[0], lat_lon[1], geog, cart, forward=True)
azi_grid_north

array([2.25714888e-03, 3.37522257e+02, 3.32342257e+02, 3.28382257e+02,
       3.24422257e+02, 3.22212257e+02, 3.19992257e+02, 3.03102257e+02,
       2.86212257e+02, 3.04622257e+02, 3.23042257e+02, 3.16102257e+02,
       3.09162257e+02, 2.90922257e+02, 2.72682257e+02, 2.69812257e+02,
       2.66952257e+02, 2.70932257e+02, 2.96912257e+02, 3.53922257e+02,
       4.09222571e+01, 5.94222571e+01, 6.79222571e+01, 6.66522571e+01,
       6.53822571e+01, 5.82922571e+01, 5.12122571e+01, 4.93822571e+01,
       4.75522571e+01, 4.15522571e+01, 3.35522571e+01, 3.42242257e+02,
       2.90922257e+02, 2.80242257e+02, 2.69562257e+02, 2.25582257e+02,
       1.81592257e+02, 1.72412257e+02, 1.63232257e+02, 1.48942257e+02,
       1.34652257e+02, 1.51292257e+02, 1.63942257e+02, 1.54752257e+02,
       1.45572257e+02, 1.49612257e+02, 1.53652257e+02, 1.60472257e+02,
       1.67292257e+02, 1.69512257e+02, 1.71722257e+02, 1.71962257e+02,
       1.72212257e+02, 1.72332257e+02, 1.72452257e+02, 1.62742257e+02,
      

In [27]:
stations_grid = stations.copy()
stations_grid[:, -1] = azi_grid_north
stations_grid

array([[0.00000000e+00, 0.00000000e+00, 2.25714888e-03],
       [1.00000000e+02, 3.90000000e-01, 3.37522257e+02],
       [2.00000000e+02, 5.30000000e-01, 3.32342257e+02],
       [3.00000000e+02, 1.15000000e+00, 3.28382257e+02],
       [4.00000000e+02, 1.77000000e+00, 3.24422257e+02],
       [5.00000000e+02, 1.82000000e+00, 3.22212257e+02],
       [6.00000000e+02, 1.88000000e+00, 3.19992257e+02],
       [7.00000000e+02, 1.43000000e+00, 3.03102257e+02],
       [8.00000000e+02, 9.70000000e-01, 2.86212257e+02],
       [9.00000000e+02, 7.20000000e-01, 3.04622257e+02],
       [1.00000000e+03, 4.70000000e-01, 3.23042257e+02],
       [1.10000000e+03, 5.70000000e-01, 3.16102257e+02],
       [1.20000000e+03, 6.60000000e-01, 3.09162257e+02],
       [1.30000000e+03, 9.60000000e-01, 2.90922257e+02],
       [1.40000000e+03, 1.25000000e+00, 2.72682257e+02],
       [1.50000000e+03, 1.47000000e+00, 2.69812257e+02],
       [1.60000000e+03, 1.66000000e+00, 2.66952257e+02],
       [1.70000000e+03, 1.66000

In [28]:
plog_grid = position_log(stations_grid, tie_in, report_raw=False)

In [29]:
proj = prj.Project(geog, cart)

In [30]:
surf_cart = proj.forward([lat_lon])
surf_cart

array([[857836.92737034, 836770.97991896]])

In [31]:
bott_off = plog_grid[-1, 3:5]
bott_off

array([4898.44948983,  147.24760863])

In [32]:
bott_cart = surf_cart + bott_off
bott_cart

array([[862735.37686018, 836918.22752758]])

In [33]:
bott_lat_lon = proj.inverse(bott_cart)[0]
bott_lat_lon

array([  31.98480975, -104.08491805])

#### Comparing diff to TDM.  We see TDM uses inverse projection method

In [34]:
tdm_bott_lat_lon = srv.iloc[-1][['LAT', 'LONG']].to_numpy()
tdm_bott_lat_lon

array([31.98481, -104.084918], dtype=object)

In [35]:
print(tdm_bott_lat_lon - bott_lat_lon)

[2.4982943358509146e-07 5.3304745506466134e-08]


#### See how many feet we we are off in projected coords TDM to LMP

In [36]:
bott_lmp = proj.forward([lmp[-1]])
bott_lmp

array([[862737.40253209, 836918.29397695]])

In [37]:
bott_cart - bott_lmp

array([[-2.02567191, -0.06644936]])

#### Try our method

In [38]:
frwd = cvt.forward_lat_lon(nev_off, lat_lon[0], lat_lon[1], geog, cart, true_north=True)
frwd

array([[  31.971339  , -104.08486   ,    0.        ],
       [  31.97133985, -104.08486045,   99.9992278 ],
       [  31.97134181, -104.08486164,  199.99598437],
       [  31.97134521, -104.08486417,  299.98475721],
       [  31.97135088, -104.08486899,  399.95182976],
       [  31.97135761, -104.08487529,  499.90276851],
       [  31.97136432, -104.0848821 ,  599.85065384],
       [  31.97136945, -104.08488908,  699.80927045],
       [  31.97137179, -104.08489517,  799.78737395],
       [  31.9713733 , -104.08489952,  899.77660105],
       [  31.97137511, -104.08490205,  999.77121768],
       [  31.97137694, -104.08490403, 1099.76709655],
       [  31.97137885, -104.08490666, 1199.76133962],
       [  31.97138056, -104.08491069, 1299.7513942 ],
       [  31.97138135, -104.08491676, 1399.7329968 ],
       [  31.97138126, -104.08492441, 1499.70477739],
       [  31.97138078, -104.0849332 , 1599.6674455 ],
       [  31.97138037, -104.08494252, 1699.62551196],
       [  31.97138199, -104.

In [39]:
frwd[-1, :-1] - bott_lat_lon

array([0., 0.])

#### Try with UTM Zone 13

In [40]:
cart_utm = 4414
frwd_utm = cvt.forward_lat_lon(nev_off, lat_lon[0], lat_lon[1], geog, cart_utm, true_north=True)

In [41]:
print(frwd_utm[-1, :-1] - bott_lat_lon)

[ 1.36040451e-02 -6.42634781e-05]


In [42]:
print(tdm_bott_lat_lon - frwd_utm[-1, :-1])

[-0.01360379530279232 6.43167828826563e-05]
